# Environment setting
環境設定

In [1]:
import os
import requests
import sys
from pathlib import Path
from pprint import pprint
from typing import Optional

import pandas as pd


# determine branch, default is main
branch: str = "main"

# determine subfolder, default is None (petsard/demo/)
subfolder: Optional[str] = "use-cases"


# Check if running in Google Colab, if so, download the utils.py file from GitHub
is_colab: bool = "COLAB_GPU" in os.environ
if is_colab:
    utils_url = (
        f"https://raw.githubusercontent.com/nics-tw/petsard/{branch}/demo/utils.py"
    )
    response = requests.get(utils_url)

    if response.status_code == 200:
        with open("utils.py", "w") as f:
            f.write(response.text)

        Path("__init__.py").touch()
    else:
        raise RuntimeError(
            f"Failed to download utils.py. Status code: {response.status_code}"
        )


# If not colab, and also contains subfolderl, add the correct path of util.py
else:
    if subfolder:
        sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
# Now import and run the setup
from utils import (
    get_yaml_path,
    setup_environment,
)


setup_environment(
    is_colab,
    branch,
    benchmark_data=[
        "adult-income",
    ],
    subfolder=subfolder,
)

In [3]:
from petsard import Executor

# YAML Configuration for PETsARD
PETsARD 的 YAML 設定

## Data Constraining
資料約束

In [4]:
yaml_file_case: str = "data-constraining.yaml"

yaml_path_case: str = get_yaml_path(
    is_colab=is_colab,
    yaml_file=yaml_file_case,
    branch=branch,
    subfolder=subfolder,
)

Configuration content:
---
Loader:
  data:
    filepath: 'benchmark/adult-income.csv'
Preprocessor:
  demo:
    method: 'default'
Synthesizer:
  demo:
    method: 'default'
Postprocessor:
  demo:
    method: 'default'
Constrainer:
  demo:
    nan_groups:
      # Delete entire row when workclass is NA
      workclass: 'delete'
      # Set income to NA if occupation is NA
      occupation:
        'erase':
          - 'income'
      # Copy educational-num value to age when educational-num exists but age is NA
      age:
        'copy':
          'educational-num'
    field_constraints:
      - "age >= 18 & age <= 65"
      - "hours-per-week >= 20 & hours-per-week <= 60"
    field_combinations:
      -
        - {'education': 'income'}
        - {'Doctorate': ['>50K'], 'Masters': ['>50K', '<=50K']}
Reporter:
  output:
    method: 'save_data'
    source: 'Constrainer'
...


### Execution and Result
執行與結果

In [5]:
# Initialize and run executor
exec_case = Executor(config=yaml_path_case)
exec_case.run()

Now is petsard_Loader[data]_Preprocessor[demo]_Synthesizer[demo]_Postprocessor[demo]_Constrainer[demo] save to csv...


In [6]:
exec_case.get_result()[
    "Loader[data]_Preprocessor[demo]_Synthesizer[demo]_Postprocessor[demo]_Constrainer[demo]_Reporter[output]"
][
    "Loader[data]_Preprocessor[demo]_Synthesizer[demo]_Postprocessor[demo]_Constrainer[demo]"
]

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,45.512960,Private,105689.810637,Bachelors,12.105628,Never-married,Transport-moving,Own-child,White,Male,0.0,0.0,38.749746,United-States,<=50K
1,30.499544,Local-gov,142167.776888,HS-grad,9.944627,Married-civ-spouse,Prof-specialty,Not-in-family,White,Female,0.0,0.0,35.750810,United-States,<=50K
2,30.666020,Private,70682.324934,Masters,11.984639,Never-married,Adm-clerical,Husband,White,Male,0.0,0.0,35.629290,United-States,<=50K
3,35.605134,Self-emp-inc,217418.285282,Some-college,8.728909,Divorced,Other-service,Own-child,White,Male,0.0,0.0,36.473107,United-States,<=50K
4,31.010968,Private,85354.009609,Bachelors,11.887583,Married-civ-spouse,Sales,Husband,White,Male,0.0,0.0,51.820506,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28553,45.744603,Local-gov,204517.753557,Assoc-acdm,10.556458,Divorced,Craft-repair,Own-child,Black,Male,0.0,0.0,50.428579,United-States,>50K
28554,40.147349,Private,215714.725316,HS-grad,8.935872,Never-married,Craft-repair,Unmarried,White,Male,0.0,0.0,47.407419,United-States,<=50K
28555,37.076569,Private,300336.661199,Doctorate,11.688683,Never-married,Handlers-cleaners,Husband,White,Male,0.0,0.0,51.664408,United-States,>50K
28556,20.479069,Private,283649.188177,Some-college,12.262464,Married-civ-spouse,Exec-managerial,Own-child,White,Female,0.0,0.0,34.931740,United-States,<=50K
